loading packages used

In [605]:
import tkinter as tk
import customtkinter as ctk
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from tkinter import ttk



In [606]:
app = tk.Tk()

if app.call("tk", "scaling") == 0:
    # "dark" for MacOS
    appearance_mode = "light"
else:
    # "light" for Windows
    appearance_mode = "dark"

app.destroy()


code for the model

In [607]:
def load_and_process_dataset():
    #loading the dataset
    df = pd.read_csv('IndianWeatherRepository.csv',index_col='last_updated',parse_dates=True)
    col_to_remove=['country','latitude','longitude','timezone','last_updated_epoch','temperature_celsius','wind_mph','wind_direction','pressure_in','precip_in','feels_like_celsius','feels_like_fahrenheit','visibility_miles','gust_mph','uv_index','gust_kph','sunrise','sunset','moonrise','moonset','moon_phase','moon_illumination','location_name','air_quality_Carbon_Monoxide', 'air_quality_Ozone','air_quality_Nitrogen_dioxide','air_quality_Sulphur_dioxide','air_quality_PM2.5','air_quality_PM10','air_quality_us-epa-index','air_quality_gb-defra-index']
    df = df.drop(columns=col_to_remove)
    condition=df['region']=='Andaman and Nicobar Islands'
    df = df[condition]
    #identify and remove duplicated rows
    duplicated_rows=df.duplicated(keep=False)
    duplicate_data=df[duplicated_rows]
    df=df.drop_duplicates(keep='first')
    #convert the int64 data to float64
    df['humidity'] = df['humidity'].astype('float64')
    df['wind_degree'] = df['wind_degree'].astype('float64')
    df['cloud'] = df['cloud'].astype('float64')
    return df

In [608]:
def arima_model(df):

    #df = pd.DataFrame(df, index=pd.date_range('2023-08-29', periods=101, freq='D'))

    #df['last_updated'] = pd.to_datetime(df['last_updated']).dt.date
    #df.set_index('last_updated', inplace=True)
    
    global parameters
    # Select the parameters for prediction
    parameters = ['temperature_fahrenheit', 'wind_kph', 'wind_degree', 'pressure_mb',
              'precip_mm', 'humidity', 'cloud', 'visibility_km']

    # Create an empty DataFrame to store predictions
    predictions_df = pd.DataFrame()
    # Iterate over each parameter
    #temperature
    train_data=df[:-7]['temperature_fahrenheit']
    test_data=df[-7:]['temperature_fahrenheit']
    model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    predictions_df['temperature_fahrenheit'] = predictions


    #wind
    train_data=df[:-7]['wind_kph']
    test_data=df[-7:]['wind_kph']
    model = ARIMA(train_data, order=(1, 1, 1))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    predictions_df['wind_kph'] = predictions


    #wind deg
    train_data=df[:-7][ 'wind_degree']
    test_data=df[-7:][ 'wind_degree']
    model = ARIMA(train_data, order=(0, 1, 1))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    predictions_df[ 'wind_degree'] = predictions


    #pressure
    train_data=df[:-7]['pressure_mb']
    test_data=df[-7:]['pressure_mb']
    model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    predictions_df['pressure_mb'] = predictions


    #preci
    train_data=df[:-7]['precip_mm']
    test_data=df[-7:]['precip_mm']
    model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    predictions_df['precip_mm'] = predictions

    #humidity
    train_data=df[:-7]['humidity']
    test_data=df[-7:]['humidity']
    model = ARIMA(train_data, order=(1, 1, 1))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    predictions_df['humidity'] = predictions


    #cloud
    train_data=df[:-7]['cloud']
    test_data=df[-7:]['cloud']
    model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    # Print the predictions
    predictions_df['cloud'] = predictions    


    #visibility 
    train_data=df[:-7]['visibility_km']
    test_data=df[-7:]['visibility_km']
    model = ARIMA(train_data, order=(0, 0, 0))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    # Print the predictions
    predictions_df['visibility_km'] = predictions
    return predictions_df


In [609]:
def encode_normalise(data):
    global encoder 
    encoder= LabelEncoder()

    # Fit the encoder on the column and transform the values
    data['condition_text'] = encoder.fit_transform(data['condition_text'])
    data['condition_text'] = data['condition_text'].astype(int)

    # Access the encoded classes
    encoded_classes = encoder.classes_
    #print("Encoded Classes:", encoded_classes)

    # Decode the encoded values to get the original labels
    decoded_labels = encoder.inverse_transform(data['condition_text'])
    #print("Decoded Labels:", decoded_labels)

    # Select only the numerical columns
    numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns

    # Create an instance of MinMaxScaler
    scaler = MinMaxScaler()

    # Fit the scaler on the numerical columns and transform the values
    data[numerical_columns] = scaler.fit_transform(data[numerical_columns])
    return data

In [610]:
def condition_text(data):
    X = data.iloc[:, [1,3,4,5,6,7,8,9]].values
    y = data.iloc[:, 2].values

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

    # Create a decision tree classifier
    clf = tree.DecisionTreeClassifier(random_state=0)

    # Train the classifier on the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    return clf

GUI MAIN CODE

In [611]:


def run_program():
    global image
    global finaltest
    run_button.place_forget()
    #heading.pack_forget()
    #framein.pack_forget()
    heading1 = tk.Label(background_im, text="WEATHER FORECAST",font=("Trebuchet MS", 20, "bold"),fg="#005FBC",anchor="center")
    heading1.pack(side="top",fill="both",padx=20, pady=15)
    heading1.configure(bg=hex_color)
    heading1.pack()
    df = pd.DataFrame()
    predictions_df = pd.DataFrame()
    data_for_train=pd.DataFrame()
    df=load_and_process_dataset()
    predictions_df=arima_model(df)
    data=df[:-7]
    finaltest=df[-7:]
    data_for_train=encode_normalise(data)
    clf=condition_text(data_for_train)
    pred_copy=predictions_df.copy(deep=True)
    # Select only the numerical columns
    numerical_columns = predictions_df.select_dtypes(include=['float64', 'int64']).columns

    # Create an instance of MinMaxScaler
    scaler = MinMaxScaler()

    # Fit the scaler on the numerical columns and transform the values
    predictions_df[numerical_columns] = scaler.fit_transform(predictions_df[numerical_columns])
    pred=clf.predict(predictions_df)
    #print(pred)

    # Decode the encoded values to get the original labels
    decoded_labels = encoder.inverse_transform(pred)

    selected_df = pred_copy.iloc[0, [0,1,4,5]]
    today_df = pd.DataFrame()
    today_df['today'] = selected_df
    dates=finaltest.index
    
    name=pd.to_datetime(selected_df.name)
    t="Temp  \n"+str(round(today_df.iat[0, 0],2))+" F"
    h="Humidity     \n"+str(round(today_df.iat[3,0],2))+" %"
    p="Precip\n"+str(round(today_df.iat[2,0],2))+" mm"
    w="Wind \n"+str(round(today_df.iat[1,0],2))+"kph"
    
    #day = name.day_name()
    
    frame = tk.Frame(background_im, bg=hex_color)
    frame.pack(side="top",fill="both",padx=20)
    label = tk.Label(frame, text=str(dates[0])[:10],font=("Trebuchet MS", 10, "bold"),fg="#005FBC", bg=hex_color)
    label.pack()
    label = tk.Label(frame, text=decoded_labels[0].upper(),font=("Trebuchet MS",15, "bold"),fg="#005F9F", bg=hex_color)
    label.pack()

    label_frame=tk.Frame(frame, bg =hex_color)
    label_frame.pack(side=tk.LEFT)

    label = tk.Label(label_frame, text=t,font=("Open Sans",9),fg="#0073CD", bg=hex_color)
    label.pack(side=tk.LEFT)
    label = tk.Label(label_frame, text=h,font=("Open Sans",9),fg="#0073CD", bg=hex_color)
    label.pack(side=tk.LEFT)
    label = tk.Label(label_frame, text=p,font=("Open Sans",9),fg="#0073CD", bg=hex_color)
    label.pack(side=tk.LEFT)
    label = tk.Label(label_frame, text=w,font=("Open Sans",9),fg="#0073CD", bg=hex_color)
    label.pack(side=tk.LEFT)

    weather_img(decoded_labels[0],frame)
    #canvas = tk.Canvas(frame, width=200, height=200, bg=hex_color)
    #canvas.pack(side=tk.RIGHT)
    #canvas.config(highlightthickness=1, highlightbackground=hex_color)
    # Load the image
    #image = tk.PhotoImage(file="cloudy.png")
    # Add the image to the canvas
    #canvas.create_image(100, 100, image=image)
    
    framea = tk.Frame(background_im, bg=hex_color)
    framea.pack(side="top",fill="both",padx=20, pady=10)
    for i in range(3):
        framei = tk.Frame(framea, bg=hex_color, width=40,height=20)
        framei.pack(side=tk.LEFT, padx=5, pady=5, expand=True)
    
        label = tk.Label(framei, text=str(dates[i+1])[:10],font=("Trebuchet MS", 7, "bold"),fg="#005FBC", bg=hex_color)
        label.pack(pady=5)
        label = tk.Label(framei, text=decoded_labels[i+1],font=("Trebuchet MS",8,"bold"),fg="#005F9F", bg=hex_color)
        label.pack(pady=5)

    frameb = tk.Frame(background_im, bg=hex_color)
    frameb.pack(side="top",fill="both", padx=20, pady=5)
    for i in range(3):
        framej = tk.Frame(frameb, bg=hex_color, width=40,height=20)
        framej.pack(side=tk.LEFT, padx=5, pady=5, expand=True)
    
        label = tk.Label(framej, text=str(dates[i+4])[:10],font=("Trebuchet MS", 7, "bold"),fg="#005FBC", bg=hex_color)
        label.pack(pady=5)
        label = tk.Label(framej, text=decoded_labels[i+4],font=("Trebuchet MS",8,"bold"),fg="#005F9F", bg=hex_color)
        label.pack(pady=5)
    
    framek = tk.Frame(background_im, bg="white", width=40,height=20)
    framek.pack(side="top",fill="both", padx=20, pady=5)
    
    label = tk.Label(framek, text="SNEHA BABY MATHEW - 21BLC1236",font=("Arial", 7, "bold"),fg="black", bg="white")
    label.pack(pady=5)
pass




for the weather pics    

In [612]:

def weather_img(weather,frame):
    global image
    canvas = tk.Canvas(frame, width=100, height=100, bg=hex_color)
    canvas.pack(side=tk.RIGHT, padx=10)
    canvas.config(highlightthickness=1, highlightbackground=hex_color)
    # Load the image
    '''['Light rain shower', 'Cloudy', 'Moderate or heavy rain shower', 'Torrential rain shower',
      'Thundery outbreaks possible', 'Partly cloudy', 'Patchy rain possible', 'Clear', 'Light drizzle', 
      'Patchy light drizzle', 'Patchy light rain', 'Heavy rain'] '''
    if weather=="Light rain shower":
        image = tk.PhotoImage(file="light rain shower.png")
    elif weather=="Cloudy":
        image = tk.PhotoImage(file="cloudy.png")
    elif weather=="Moderate or heavy rain shower":
        image = tk.PhotoImage(file="Moderate or heavy rain shower.png")
    elif weather=="Torrential rain shower":
        image = tk.PhotoImage(file="Torrential rain shower.png")
    elif weather=="Thundery outbreaks possible":
        image = tk.PhotoImage(file="Thundery outbreaks possible.png")
    elif weather=="Partly cloudy":
        image = tk.PhotoImage(file="Partly cloudy.png")
    elif weather=="Patchy rain possible":
        image = tk.PhotoImage(file="Patchy rain possible.png")
    elif weather=="Clear":
        image = tk.PhotoImage(file="Clear.png")
    elif weather=="Light drizzle":
        image = tk.PhotoImage(file="Light drizzle.png")
    elif weather=="Patchy light drizzle":
        image = tk.PhotoImage(file="Patchy light drizzle.png")
    elif weather=="Patchy light rain":
        image = tk.PhotoImage(file="Patchy light rain.png")
    elif weather=="Heavy rain":
        image = tk.PhotoImage(file="Heavy rain.png")
    canvas.create_image(50, 50, image=image)
    pass


In [613]:
#import tkinter as tk

class BackgroundImage(tk.Canvas):
    def __init__(self, parent, image_path):
        tk.Canvas.__init__(self, parent)
        self.image = tk.PhotoImage(file=image_path)
        self.create_image(0, 0, anchor=tk.NW, image=self.image)
        self.pack(fill=tk.BOTH, expand=True)


In [614]:
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = tk.Tk()
app.geometry("400x480")
app.title("RainCheck")
# Define the RGB color (replace the numbers with your desired RGB values)
rgb_color = (158, 203, 248)
# Convert the RGB color to Hex
hex_color = "#{:02x}{:02x}{:02x}".format(*rgb_color)
# Set the background color to the Hex color
app.configure()
background_im=BackgroundImage(app, "sky.png")
#heading = tk.Label(background_im, text="WEATHER FORECAST",font=("Trebuchet MS", 20, "bold"),anchor="e")
#heading.configure(bg=hex_color)
#heading.place(relx=0.5, rely=0.4, anchor='center')
#heading.pack()

run_button = tk.Button(background_im, text="WEATHER FORECAST", font=("Trebuchet MS", 20, "bold"),fg="#005FBC",command=run_program)
run_button.configure(bg=hex_color, relief="groove")
#run_button.pack()
run_button.place(relx=0.5, rely=0.5, anchor='center')

app.mainloop()


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecas